In [1]:
import pymc as pm
import pandas as pd
import numpy as np
import arviz as az
from pymc.math import dot
import aesara.tensor as at

%load_ext lab_black
%load_ext watermark

# Brozek index prediction

This example goes over linear regression and Bayesian $R^2$.

Adapted from [unit 7: fat1.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit7/fat2d.odc), [unit 7: fat2d.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit7/fat1.odc) and [fatmulti.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit7/fatmulti.odc).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/fat.tsv).

Associated lecture videos:
Unit 7 Lesson 11

Unit 7 Lesson 13

## Problem statement

Percentage of body fat, age, weight, height, and ten body circumference measurements (e.g., abdomen) were recorded for 252 men. Percentage of body fat is estimated through an underwater weighing technique.

The data set has 252 observations and 15 variables. Brozek index (Brozek et al., 1963) is obtained by the underwater weighing while other 14 anthropometric variables are obtained using scales and a measuring tape.

- y = Brozek index
- X1 = 1 (intercept)
- X2 = age
- X3 = weight
- X4 = height
- X5 = adipose
- X6 = neck  
- X7 = chest
- X8 = abdomen
- X9 = hip
- X10 = thigh
- X11 = knee   
- X12 = ankle
- X13 = bicep
- X14 = forearm
- X15 = wrist

These anthropometric variables are less intrusive but also less reliable in assessing the body fat index.

Set a linear regression to predict the Brozek index from these body measurements.

## Single predictor (X8)

This is a recreation of fat1.odc.

In [4]:
data = pd.read_csv("../data/fat.tsv", sep="\t")

y = data["y"].to_numpy(copy=True)
X = data["X8"].to_numpy(copy=True)

# p will be the number of predictors + intercept (1 + 1 in this case)
n, p = X.shape[0], 2

In [5]:
with pm.Model() as m:
    tau = pm.Gamma("tau", 0.001, 0.001)
    beta0 = pm.Normal("beta0_intercept", 0, tau=0.001)
    beta1 = pm.Normal("beta1_abdomen", 0, tau=0.001)
    variance = pm.Deterministic("variance", 1 / tau)

    mu = beta0 + beta1 * X
    likelihood = pm.Normal("likelihood", mu=mu, tau=tau, observed=y)

    # Bayesian R2 from fat1.odc
    sse = (n - p) * variance
    cy = y - y.mean()
    sst = dot(cy, cy)
    br2 = pm.Deterministic("br2", 1 - sse / sst)

    trace = pm.sample(2000)
    ppc = pm.sample_posterior_predictive(trace)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, beta0_intercept, beta1_abdomen]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 25 seconds.


In [6]:
az.summary(trace, hdi_prob=0.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta0_intercept,-34.984,2.478,-39.805,-30.138,0.048,0.034,2708.0,2829.0,1.0
beta1_abdomen,0.583,0.027,0.531,0.635,0.001,0.000,2734.0,2859.0,1.0
tau,0.049,0.004,0.041,0.057,0.000,0.000,3688.0,3705.0,1.0
variance,20.549,1.847,17.071,24.156,0.030,0.022,3688.0,3705.0,1.0
br2,0.659,0.031,0.600,0.717,0.001,0.000,3688.0,3705.0,1.0


This matches the results from the U7 L11 video.

Another way to calculate the $R^2$ using a posterior predictive check (keeping in mind that there is no standard "Bayesian $R^2$") and the results will be slightly different:

In [7]:
# get the mean y_pred across all chains
y_pred = np.array(ppc.posterior_predictive.likelihood.mean(axis=(0, 1)))

az.r2_score(y, y_pred)

r2        0.660194
r2_std    0.000000
dtype: float64

In this case they agree, but that won't always be true.

## Multinomial regression with all predictors

Based on fat2d.odc or fatmulti.odc (they appear to be identical).

In [8]:
data = pd.read_csv("../data/fat.tsv", sep="\t")

y = data["y"].to_numpy(copy=True)
X = data.iloc[:, 1:].to_numpy(copy=True)

# add intercept
X_aug = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
n, p = X_aug.shape

# Zellner's g
g = p**2

n, p, g

(252, 15, 225)

In [9]:
X_aug.shape

(252, 15)

In [10]:
new_matrix = np.zeros((15, 15))
for i in range(15):
    for j in range(15):
        new_matrix[i, j] = np.inner(X_aug[:, i], X_aug[:, j])
new_matrix.shape

(15, 15)

In [11]:
np.equal(np.dot(X_aug.T, X_aug), new_matrix)

array([[ True,  True,  True,  True, False, False, False, False,  True,
         True, False, False, False, False, False],
       [ True,  True, False,  True, False, False, False,  True, False,
        False, False, False,  True, False, False],
       [ True, False,  True,  True, False, False,  True,  True, False,
        False, False, False,  True, False, False],
       [ True,  True,  True,  True, False, False, False,  True, False,
        False, False, False, False, False, False],
       [False, False, False, False, False, False,  True, False, False,
        False, False, False, False, False,  True],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False,  True],
       [False, False,  True, False,  True, False, False, False, False,
        False, False, False, False, False, False],
       [False,  True,  True,  True, False, False, False, False,  True,
         True, False, False, False, False,  True],
       [ True, False, Fa

In [12]:
np.allclose(np.dot(X_aug.T, X_aug), new_matrix)

True

In [13]:
np.dot(X_aug.T, X_aug).shape

(15, 15)

In [14]:
mu_beta = np.zeros(p)

In [15]:
with pm.Model() as m2d:
    tau = pm.Gamma("tau", 0.01, 0.01)
    variance = pm.Deterministic("variance", 1 / tau)

    tau_matrix = at.fill(at.zeros((15, 15)), tau)
    tau_beta = tau_matrix / g * dot(X_aug.T, X_aug)
    beta = pm.MvNormal("beta", mu_beta, tau=tau_beta)

    mu = dot(X_aug, beta)
    pm.Normal("likelihood", mu=mu, tau=tau, observed=y)

    # Bayesian R2 from fat2d.odc
    sse = (n - p) * variance
    cy = y - y.mean()
    sst = dot(cy, cy)
    br2 = pm.Deterministic("br2", 1 - sse / sst)
    br2_adj = pm.Deterministic("br2_adj", 1 - (n - 1) * variance / sst)

    trace = pm.sample(1000)
    ppc = pm.sample_posterior_predictive(trace)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, beta]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 1080 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.


In [16]:
az.summary(trace, hdi_prob=0.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta[0],-15.675,16.626,-48.860,17.231,0.412,0.291,1628.0,2124.0,1.0
beta[1],0.056,0.031,-0.002,0.118,0.001,0.000,2461.0,2704.0,1.0
beta[2],-0.083,0.051,-0.182,0.017,0.001,0.001,1676.0,1974.0,1.0
beta[3],-0.052,0.110,-0.268,0.156,0.002,0.002,2182.0,2649.0,1.0
beta[4],0.059,0.290,-0.532,0.613,0.006,0.005,2278.0,2253.0,1.0
beta[5],-0.440,0.226,-0.886,0.002,0.004,0.003,3173.0,2873.0,1.0
beta[6],-0.029,0.100,-0.224,0.163,0.002,0.002,2398.0,2474.0,1.0
beta[7],0.877,0.090,0.697,1.054,0.002,0.001,2559.0,2549.0,1.0
beta[8],-0.202,0.140,-0.481,0.060,0.003,0.002,2238.0,2965.0,1.0
beta[9],0.227,0.138,-0.038,0.504,0.003,0.002,2974.0,2888.0,1.0


In [17]:
y_pred = np.array(ppc.posterior_predictive.likelihood.mean(axis=(0, 1)))

az.r2_score(y, y_pred)

r2        0.747027
r2_std    0.000000
dtype: float64

Reading on g-priors

https://arxiv.org/abs/1702.01201
https://towardsdatascience.com/linear-regression-model-selection-through-zellners-g-prior-da5f74635a03
https://en.wikipedia.org/wiki/G-prior\

original paper:

Zellner, A. (1986). "On Assessing Prior Distributions and Bayesian Regression Analysis with g Prior Distributions". In Goel, P.; Zellner, A. (eds.). Bayesian Inference and Decision Techniques: Essays in Honor of Bruno de Finetti. Studies in Bayesian Econometrics and Statistics. Vol. 6. New York: Elsevier. pp. 233–243. ISBN 978-0-444-87712-3.

In [18]:
%watermark -n -u -v -iv -p aesara,aeppl

Python implementation: CPython
Python version       : 3.10.4
IPython version      : 8.4.0

aesara: 2.6.6
arviz : 0.12.1
numpy : 1.22.4
pandas: 1.4.2
pymc  : 4.0.0

